# Clustering Neighbourhoods in Toronto
## Coursera Capstone Project  - Parts 1 and 2

### Harishankar


### Part 1 - Converting Wikipedia html table into a DataFrame

In [1]:
from bs4 import BeautifulSoup as bsoup
from urllib.request import urlopen as uReq
import requests
import lxml
import pandas as pd
from pandas import DataFrame
import numpy as np

In [2]:
my_url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M' 

### Parsing the web html file with BeautifulSoup package

In [3]:
r=requests.get(my_url)
page=bsoup(r.text,"html.parser")
rtable=page.table

In [6]:
results=rtable.find_all('tr')
nrows=len(results)
nrows

290

In [8]:
header=results[0].text.split()
header

['Postcode', 'Borough', 'Neighbourhood']

### Let's check some  rows in order to prepare to build the loop that will extract all cells into a DataFrame.  For example, let's examine row 85


In [9]:
results[85].text

'\nM5J\nDowntown Toronto\nToronto Islands\n'

In [10]:
results[85].text.split('\n')

['', 'M5J', 'Downtown Toronto', 'Toronto Islands', '']

In [11]:
results[85].text.split('\n')

['', 'M5J', 'Downtown Toronto', 'Toronto Islands', '']

In [12]:
Postcode=results[85].text.split('\n')[1]
Postcode

'M5J'

In [13]:
Borough=results[85].text.split('\n')[2]
Borough

'Downtown Toronto'

In [14]:
Neighborhood=results[85].text.split('\n')[3]
Neighborhood

'Toronto Islands'

## Iteration loop to extract all cells into a dataframe df

In [15]:
# iteration loop to harvest all records

records =[]
n=1
while n < nrows :
    Postcode=results[n].text.split('\n')[1]
    Borough=results[n].text.split('\n')[2]
    Neighborhood=results[n].text.split('\n')[3]
    records.append((Postcode, Borough,Neighborhood))
    n=n+1

df=pd.DataFrame(records, columns=['PostalCode', 'Borough', 'Neighbourhood'])
df.head(5)



,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Initial Dataframe has 289 cells or rows of data, as expected. Great!

In [18]:
# How many rows have Borough equal to 'Not assigned'?
df[df['Borough']=='Not assigned'].count()

PostalCode       77
Borough          77
Neighbourhood    77
dtype: int64

In [19]:
# drops those rows where 'Not assigned' appears in column '[Borough]'
df1=df[~df.Borough.str.contains("Not assigned")]
df1=df1.reset_index(drop=True)

In [20]:
df1.head(5)

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


### Now,  let's replace  'Not assigned' neighborhoods with the name of the Borough


In [23]:
df1.loc[df1['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = df1['Borough']

In [24]:
df1.head()           # Now, there are no rows with 'Not assigned' strings anymore

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


### Let's examine de dataframe first to set up the best loop algorithm. How many unique postalcodes , Boroughs and Neighbourhood names are there?



In [26]:
postalcodes = df1['PostalCode'].nunique()
boroughs = df1['Borough'].nunique()
neighbourhoods= df1['Neighbourhood'].nunique()
print('Unique Postalcodes : ' + str(postalcodes))
print('Unique Boroughs  : '+ str(boroughs))
print('Unique Neighbourhoods  :' + str(neighbourhoods))

     

Unique Postalcodes : 103
Unique Boroughs  : 11
Unique Neighbourhoods  :210


## Let's Consolidate the dataframe to each unique PostalCodes and aggregated Neighbourhoods

In [28]:
# Starting DataFrame df1
df1.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [29]:
nrows1=len(df1)
nrows1

212

### Let's keep df1 intact and let's work with df2 as precaution

In [30]:
# DataFrame df2 also has 212 rows or cells
df2=df1
df2.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [32]:
# I will use two pointers, 'n and m' to sweep through the dataframe. So 'n' will vary from
#  0 to 211, and 'm ' will vary from 1 to 212
nrows2=len(df2)-1
nrows2

211

## Loop Algorithm to extract the consolidate dataframe df2 with unique postalcodes.
### We expect a df2 with 103 rows because that is the number of unique postalcodes

In [33]:
n=0

while n < nrows2 :
    post1=df2.iloc[n,0]
    #post1
    m=n+1
    post2=df2.iloc[m,0]
    #post2
    neigh1=df2.iloc[n,2]
    neigh2=df2.iloc[m,2]
    if post1==post2:
        df2.Neighbourhood[n,2] = neigh1=neigh1+','+neigh2
        #df2 = df2[df2.Neighbourhood != 'neigh2']
        df2=df2.drop(df2.index[m])
        nrows2=nrows2-1
        df2 = df2.reset_index(drop=True)
    else:
        n=n+1


#df2 = df2.reset_index(drop=True)
# print(post1, post2, 'Nigh1 is...'+ neigh1,',,,,,,,','Nigh2 is...' + neigh2, n,m)
df2.index


RangeIndex(start=0, stop=103, step=1)

In [34]:
df2.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Kingsway Park South West,Mimico NW,The Queensw..."
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [35]:
df2.index

RangeIndex(start=0, stop=103, step=1)

## DataFrame consolidated to unique 103 PostalCodes 

In [36]:
df2.head(15)

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Kingsway Park South West,Mimico NW,The Queensw..."
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge,Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens,Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson,Garden District"


### End of Part 1

## PART 2: Now, let's get the Latitude and Longitude coordenates for every PostalCode and add those columns to the dataframe

In [1]:
import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

#import geocoder 

In [39]:
# Read the cvs file and convert it to a dataframe

url='http://cocl.us/Geospatial_data'
df_pcodes=pd.read_csv(url)
df_pcodes.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [41]:
df_pcodes.columns = ['PostalCode', 'Latitude', 'Longitude']
df2s=df2.sort_values('PostalCode')
df2s.head()

,PostalCode,Borough,Neighbourhood
6,M1B,Scarborough,"Rouge,Malvern"
12,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
18,M1E,Scarborough,"Guildwood,Morningside,West Hill"
22,M1G,Scarborough,Woburn
26,M1H,Scarborough,Cedarbrae


In [42]:
neighborhoods=pd.merge(df2s,df_pcodes, how='right', on = 'PostalCode')
neighborhoods.head(15)

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848


### END OF PART 2